# nn.Embedding

주로 자연어 처리(NLP) 작업에서 단어를 dense vector로 변환하는 데 사용되는 레이어이다.<br>
고유한 정수 인덱스를 통해 단어를 사전에 매핑하고, 각 단어를 고정된 크기의 실수 벡터로 변환하여 모델이 처리할 수 있도록 수행한다.

위의 설명과 같이 `nn.Embedding`은 lookup table 형식이다. <br>
벡터가 들어오면 벡터가 나가는 `nn.Linear`와 다르게 `nn.Embedding`은 정수 인덱스가 들어오면 해당 **인덱스-행**에 해당하는 실수 벡터를 반환해준다.

이를 이해하기 위해선 NLP에서 모델의 입력에 어떤 값이 들어가는지를 이해해야 한다. <br>
따라서 간단한 토크나이저를 불러와 모델의 입력을 만들어보겠다.

# Tokenizer

Tokenizer 간단히 살펴보기

In [3]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

input_sentence = "안녕하세요 문자열이 어떻게 변할까요?"

print(tokenizer(input_sentence))

{'input_ids': [101, 1463, 30006, 30021, 29992, 30010, 30025, 30005, 30006, 29997, 30009, 29999, 30013, 1459, 30014, 30021, 30000, 30006, 29999, 30010, 30022, 29999, 30019, 100, 100, 1029, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


위와 같이 자연어 문장에 대해 `tokenizer`를 적용하는 경우 문자열을 정수 토큰 형태로 변환한다.<br>
출력으로 나온 딕셔너리를 볼 수 있는데, 여기서 문장이 정수로 변환된 것은 `input_ids`이다.

'input_ids'를 만드는 `tokenizer`의 작동 순서는 다음과 같다.

1. 문장을 토큰 단위로 나눈다.
2. 단어사전(vocabulary)으로 부터 각 토큰의 고유 인덱스를 할당한다.
3. 이를 `input_ids`로 반환한다.

이렇게 정수 인덱스로 매핑된 문장의 토큰들을 `nn.Embedding`이 **lookup table** 형식으로 지정된 크기에 맞게 각각에 벡터를 반환하게 된다. <br>
따라서 `nn.Embedding`의 행 개수는 단어사전의 개수만큼 할당이 되어야한다. (그렇지 않으면 벡터로 변환되지 못하는 값이 존재함)

따라서 토크나이저의 단어사전 크기를 확인한 뒤, 임베딩 테이블에게 파라미터로 주어야한다.

In [9]:
vocab_size = tokenizer.vocab_size
h_dim = 768

emb = nn.Embedding(vocab_size, h_dim)

input_ids = tokenizer(input_sentence)['input_ids']
emb(input_ids)

위와 같이 가장 많이 사용되는 `nn.Embedding`의 파라미터는 `num_embeddings`와 `embedding_dim`이다.

`num_embeddings`는 임베딩 사전의 크기, 룩업 테이블의 행수를 의미한다.<br>
`embedding_dim`은 각 토큰에 할당할 벡터 사이즈(각 임베딩 벡터의 차원수)를 의미한다.

이제 만들어진 임베딩 레이어로부터 만들어진 토큰들이 어떻게 변하는지 코드로 확인하겠다.

In [21]:
input_ids = tokenizer(input_sentence)['input_ids']
input_ids = torch.tensor(input_ids) # torch tensor 형태로 변환해야 함
print(f"변환 전 토큰 크기 : {input_ids.shape}")


embedding_vector = emb(input_ids)
print(embedding_vector)
print(f"변환 후 임베딩 벡터 크기 : {embedding_vector.shape}")

변환 전 토큰 크기 : torch.Size([27])
tensor([[ 1.3857,  1.1258,  0.5728,  ..., -1.0338,  0.5575, -0.5610],
        [-0.2785,  1.3419, -1.1276,  ...,  0.3262,  0.1900,  1.2077],
        [ 0.0083,  0.5585, -1.9414,  ..., -0.8100, -0.9008,  0.4597],
        ...,
        [ 0.2617,  0.2667,  0.9521,  ..., -0.2224, -0.7953, -0.6154],
        [-0.1822,  0.1741,  0.2749,  ...,  0.0811, -2.2635, -0.8073],
        [-1.5654,  0.2975,  1.1201,  ..., -0.1141, -1.4782,  0.5185]],
       grad_fn=<EmbeddingBackward0>)
변환 후 임베딩 벡터 크기 : torch.Size([27, 768])


위의 결과값에서 볼 수 있듯이 각 토큰의 정수 인덱스에 해당하는 768 차원의 벡터가 할당되어 임베딩 벡터가 생성됨을 확인 할 수 있다.

토큰의 정수 인덱스에 해당하는 벡터 자체를 불러오는게 맞는지를 아래의 코드를 통해 알아보겠다.

In [35]:
idx = input_ids[0].item() # 첫번째 토큰의 인덱스
idx_vector = emb.weight[idx] # 첫번째 토큰의 인덱스에 해당하는 임베딩 벡터 

print("일치함" if all(idx_vector == embedding_vector[0]) else "일치하지 않음") # 임베딩된 벡터와 일치하는지 확인

일치함


In [38]:
embedding = nn.Embedding(5, 10)
comp = next(iter(embedding.named_parameters()))[1].data[2]
test = torch.tensor([2])
print(embedding(test)==comp)

tensor([[True, True, True, True, True, True, True, True, True, True]])


위와 같이 `nn.Embedding`은 룩업 테이블 형식임을 확인할 수 있다.<br>
또 그러한 특성상 당연히 `bias`는 존재하지 않는다.

# 추가 파라미터

Document에 나와있는 추가 파라미터(optional parameter)는 다음과 같다.


- `padding_idx`

padding_idx에 있는 항목은 gradient에 기여하지 않으며 훈련 중에 업데이트되지 않고 고정된 값으로 남는다. <br>
기본적으로 padding_idx에 있는 임베딩 벡터는 0으로 초기화되지만, 다른 값으로 업데이트하여 패딩 벡터로 사용할 수 있다.


- `max_norm`

max_norm보다 큰 노름을 가진 각 임베딩 벡터는 max_norm으로 재정규화한다.


- `norm_type` 

max_norm 옵션에서 계산할 p-노름의 값을 나타낸다. (기본값은 2로, L2 노름을 의미)<br>


- `scale_grad_by_freq`

미니 배치 내에서 단어의 빈도에 대한 역수를 사용해 기울기를 스케일링한다. (기본값은 False)<br>


- `sparse`

True인 경우, 가중치 행렬에 대한 기울기는 희소 텐서로 계산된다.<br>